## Pipelines QC code

In [325]:
import pygsheets # use 'pip install pygsheets'
import numpy

import pandas as pd
import geopandas as gpd
from geopandas.tools import overlay

from shapely.geometry import Point, LineString, MultiLineString, Polygon
from shapely.ops import cascaded_union
from shapely import wkt

import the google spreadsheet as a pandas dataframe

In [326]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')

spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet[1].get_as_df()
oil_pipes = spreadsheet[3].get_as_df()
owners = spreadsheet[2].get_as_df()

In [327]:
# from https://www.marineregions.org/downloads.php
# in the section "Marine and land zones: the union of world country boundaries and EEZ's"
eez_file = '../data/EEZ_land_union_v3_202003/EEZ_Land_v3_202030.shp'

In [328]:
status_in_dev = ['proposed', 'construction', 'shelved']

In [329]:
#gas_pipes = gas_pipes.dropna(subset=['ProjectID'], how='any')

## clean up
#gas_pipes = gas_pipes.rename(columns={'SegmentName': 'SegmentName',})
#gas_pipes['Segment name'] = gas_pipes['Segment name'].fillna('')
gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column

In [330]:
no_route_options = [
    'Unavailable', 
    'Capacity expansion only', 
    'Bidirectionality upgrade only',
    ''
]

# filter for in-development only
gas_pipes = gas_pipes[gas_pipes['Status'].str.lower().isin(status_in_dev)]

In [331]:
oil_pipes = oil_pipes.dropna(subset=['ProjectID'])

# exclude oil rows with "don't add" for ProjectID
oil_pipes = oil_pipes[oil_pipes['ProjectID']!="don't add"]

### check IDs
Does each ProjectID correspond to one combination of "Pipeline name" + "Segment name"?

In [332]:
def test_project_ids_unique(df):
    id_counts = df.groupby(['ProjectID'])['ProjectID'].count()
    id_count_multi = id_counts[id_counts > 1]
    if len(id_count_multi)==0:
        print("Test passed!")
    else:
        print(f"Test failed! Not all ProjectIDs were unique; there were {len(id_count_multi)} problems.")
        print(id_count_multi)

In [333]:
def test_each_projectid_matches_one_pipeline_segment_name(df):
    df['ComboName'] = (df['PipelineName'] + ' ' + df['SegmentName'].astype(str)).str.strip()

    df = df[['ProjectID', 'ComboName']]
    counts = df.groupby('ProjectID')['ComboName'].count()
    multiname = counts[counts > 1]

    if len(multiname)==0:
        print("Test passed!")
    else:
        print("Error!" + f" Problem with IDs; some ProjectID values matched more than one Pipeline name + Segment name:")
        print(multiname)

In [334]:
test_project_ids_unique(gas_pipes)
test_each_projectid_matches_one_pipeline_segment_name(gas_pipes)

Test passed!
Test passed!


## check names
* Is each Pipeline name + Segment name a unique combination?
* Does each pipeline name correspond to only one wiki page?
* Does each wiki page corresond to only one pipeline name?

In [335]:
def test_names_unique(df):
    # are the names unique? (Combination of 'PipelineName' & 'SegmentName')
    df['ComboName'] = (df['PipelineName'] + ' ' + df['SegmentName'].astype(str)).str.strip()
    id_counts = df.groupby(['ComboName'])['ComboName'].count()
    id_count_multi = id_counts[id_counts > 1]
    
    if len(id_count_multi)==0:
        print("Test passed! All names unique.")
    else:
        print("Test failed!" + f" There were names repeated:")
        keep_cols = ['Countries', 'PipelineName', 'SegmentName', 'ProjectID']
        print(df[df['ComboName'].isin(id_count_multi.index)][keep_cols].sort_values(by=keep_cols))

In [336]:
def check_per_wiki_only_one_pipeline_name(df):
    df['Wiki'] = df['Wiki'].str.split('#').str[0]
    df = df[['PipelineName', 'Wiki']].drop_duplicates()
    
    wiki_counts = df.groupby(['Wiki'])[['Wiki']].count()
    wiki_count_multi = wiki_counts[wiki_counts['Wiki'] > 1]
    df2 = df[df['Wiki'].isin(wiki_count_multi.index)]
    
    if len(df2)==0:
        print("Test passed!")
    else:
        print(f"Test check_per_wiki_only_one_pipeline_name failed!")
        print()
        for wiki in df2['Wiki'].unique().tolist():
            print(f"For wiki {wiki} there were the following pipeline names:")
            print(df2[df2['Wiki']==wiki]['PipelineName'].tolist())
            print()

In [337]:
def check_per_pipeline_name_only_one_wiki(df):
    df['Wiki'] = df['Wiki'].str.split('#').str[0]
    df = df[['PipelineName', 'Wiki']].drop_duplicates()
    
    name_counts = df.groupby(['PipelineName'])[['PipelineName']].count()
    name_count_multi = name_counts[name_counts['PipelineName'] > 1]
    df2 = df[df['PipelineName'].isin(name_count_multi.index)]
    
    if len(df2)==0:
        print("Test passed!")
    else:
        print(f"Test check_per_pipeline_name_only_one_wiki failed!")
        print()
        for name in df2['PipelineName'].unique().tolist():
            print(f"For pipelineName {name} there were the following wiki URLs:")
            print(df2[df2['PipelineName']==name]['Wiki'].tolist())
            print()

In [338]:
test_names_unique(gas_pipes)

Test passed! All names unique.


In [339]:
check_per_wiki_only_one_pipeline_name(gas_pipes)

Test check_per_wiki_only_one_pipeline_name failed!

For wiki https://www.gem.wiki/Sichuan%E2%80%93Shanghai_Parallel_Gas_Pipeline there were the following pipeline names:
['Sichuan–Shanghai Parallel Gas Pipeline', 'Sichuan-Shanghai Parallel Gas Pipeline']



In [340]:
check_per_pipeline_name_only_one_wiki(gas_pipes)

Test passed!


## check wiki address

In [341]:
# TO DO: Find cases in which pipeline name doesn't match wiki URL: To do comparison, need to handle unicode characters in URLs, turning them into regular characters.
#(Or, turning the pipeline name characters into unicode.)

# also check for format of wiki pages, should be https://gem.wiki.com/PipelineName - if it's any different (e.g., has a #), flag it

## check owners

In [342]:
# Check ProjectIDs in "Pipelines" sheet vs "Owners" sheet
# Any that are in one sheet and not the other?
#owners = owners.dropna(subset=['ProjectID'])

In [343]:
gas_pipes_projectids = gas_pipes['ProjectID'].tolist()
oil_pipes_projectids = oil_pipes['ProjectID'].tolist()
all_pipes_projectids = gas_pipes_projectids + oil_pipes_projectids
owners_projectids = owners['ProjectID'].tolist()

In [344]:
in_pipelines_not_owners = [x for x in all_pipes_projectids if x not in owners_projectids]
print(f"There were {len(in_pipelines_not_owners)} PipelineIDs in main sheets (gas & oil) that were not in Owners sheet:")
print(in_pipelines_not_owners)

There were 0 PipelineIDs in main sheets (gas & oil) that were not in Owners sheet:
[]


In [345]:
# in_owners_not_pipelines = [x for x in owners_projectids if x not in all_pipes_projectids]
# print(f"There were {len(in_owners_not_pipelines)} PipelineIDs in Owners sheet that were not in main sheets (gas & oil):")
# print(in_owners_not_pipelines)

## check countries
* Check entries in "Countries” column: Does it contain both the “StartCountry” and “EndCountry” entries?

In [346]:
def check_that_start_end_countries_in_countries_column(df):   
    missing_country = df[(df['StartCountry'].isna()) | (df['EndCountry'].isna())]
    if len(missing_country)>0:
        print("Error! There were missing entries for Start/End Country:")
        print(missing_country[['ProjectID', 'Countries', 'StartCountry', 'EndCountry']])
        print()
    
    for row in df.index:
        try:
            countries_list = df.at[row, 'Countries'].split(',')
            countries_list = [x.strip() for x in countries_list]
            start_country = df.at[row, 'StartCountry']
            end_country = df.at[row, 'EndCountry']
            project_id = df.at[row, 'ProjectID']
            if pd.isna(start_country)==False and start_country not in countries_list:
                print(f"Error! For {project_id}, StartCountry {start_country} not in countries list: {countries_list}")
            if pd.isna(end_country)==False and end_country not in countries_list:
                print(f"Error! For {project_id}, EndCountry {end_country} not in countries list: {countries_list}")
        except:
            print(f"Hit exception: df.at[row, 'Countries']: {df.at[row, 'Countries']}")

In [347]:
check_that_start_end_countries_in_countries_column(gas_pipes)

In [348]:
def same_state_province_as_country(df):
    """
    Look for those with state same as country.
    """
    same_start = df[df['StartState/Province']==df['StartCountry']]
    if len(same_start)>0:
        print("Some had the same start state/province as country; may be incorrect")
        print(same_start[['ProjectID', 'StartState/Province', 'StartCountry']].sort_values(by=['StartCountry', 'StartState/Province']))
        print()

    same_end = df[df['EndState/Province']==df['EndCountry']]
    if len(same_end)>0:
        print("Some had the same end state/province as country; may be incorrect")
        print(same_end[['ProjectID', 'EndState/Province', 'EndCountry']].sort_values(by=['EndCountry', 'EndState/Province']))
        print()

In [349]:
same_state_province_as_country(gas_pipes)

In [350]:
# look for outliers in location details
def location_outliers(df_arg):
    print("Check start state/province")
    df = df_arg.copy()
    # exclude those with same state/province as country
    df = df[df['StartState/Province']!=df['StartCountry']]
    df['StartState/Province'] = df['StartState/Province'].fillna('')
    df['Start_State_Country'] = df['StartState/Province'] + ', ' + df['StartCountry']
    df['Start_State_Country'] = df['Start_State_Country'].str.strip(', ')

    counts = df['Start_State_Country'].value_counts()
    singletons = counts[counts==1]
    sel_df = df[df['Start_State_Country'].isin(singletons.index)]
    for country in sel_df['StartCountry'].sort_values().unique().tolist():
        singleton_states = sel_df[sel_df['StartCountry']==country]['StartState/Province'].tolist()
        singleton_states = [x for x in singleton_states if x != '']
        singleton_states = list(set(singleton_states))
        singleton_states.sort()
        if len(singleton_states)>0:
            print(f"For country {country}, singleton states: {singleton_states}")
            print()
    
    print("------------------------")
    print("Check end state/province")
    
    df = df_arg.copy()
    # exclude those with same state/province as country
    df = df[df['EndState/Province']!=df['EndCountry']]
    df['EndState/Province'] = df['EndState/Province'].fillna('')
    df['End_State_Country'] = df['EndState/Province'] + ', ' + df['EndCountry']
    df['End_State_Country'] = df['End_State_Country'].str.strip(', ')
    
    counts = df['End_State_Country'].value_counts()
    singletons = counts[counts==1]
    sel_df = df[df['End_State_Country'].isin(singletons.index)]
    for country in sel_df['EndCountry'].sort_values().unique().tolist():
        singleton_states = sel_df[sel_df['EndCountry']==country]['EndState/Province'].tolist()
        singleton_states = [x for x in singleton_states if x != '']
        singleton_states = list(set(singleton_states))
        singleton_states.sort()
        if len(singleton_states)>0:
            print(f"For country {country}, singleton states: {singleton_states}")
            print()

In [351]:
location_outliers(gas_pipes)

Check start state/province
For country Albania, singleton states: ['Fier']

For country Algeria, singleton states: ['Laghouat', 'Tlemcen']

For country Argentina, singleton states: ['Buenos Aires', 'Formosa', 'Jujuy', 'Santa Cruz']

For country Australia, singleton states: ['Northern Territory']

For country Bolivia, singleton states: ['Santa Cruz']

For country Bosnia and Herzegovina, singleton states: ['Dalmatia']

For country Brazil, singleton states: ['São Paulo']

For country Canada, singleton states: ['Nova Scotia']

For country Chile, singleton states: ['Atacama']

For country China, singleton states: ['Heilongjiang', 'Liaoning', 'Qinghai']

For country Colombia, singleton states: ['Bolivar', 'Córdoba', 'Valle del Cauca']

For country Djibouti, singleton states: ['Somali']

For country Egypt, singleton states: ['Port Said']

For country Ghana, singleton states: ['Western']

For country Greece, singleton states: ['Eastern Macedonia and Thrace']

For country Hungary, singleton sta

In [352]:
# can also look like this to find typos and other small differences
sel_country = 'Russia'
gas_pipes[gas_pipes['StartCountry']==sel_country]['StartState/Province'].value_counts().sort_index()

                                  5
Altai Krai                        3
Irkutsk Oblast                    1
Khabarovsk Krai                   1
Komi                              1
Leningrad Oblast                  1
Murmansk Oblast                   1
Nizhgorod Oblast                  1
Republic of Komi                  1
Republic of Sakha (Yakutia)       1
Respublic of Sakha (Yakutia)      1
Vologda Oblast                    1
Yamalo-Nenets Autonomous Okrug    2
Yamalo-Nenets Autonomus Okrug     1
Name: StartState/Province, dtype: int64

## check routes
* Check routes vs StartCountry & EndCountry

In [353]:
df = gas_pipes.copy()
df = df[~df['Route'].isin(no_route_options)] # remove any capacity expansions only, etc.
#df = df.dropna(subset=['Route']) # drop all missing routes
route_start = df['Route'].str.split(':').str[-1]
route_end = df['Route'].str.split(':').str[-1]

In [354]:
for row in df.index:
    start_country = df.at[row, 'StartCountry']
    end_country = df.at[row, 'EndCountry']

In [355]:
def read_eez_file(eez_file):
    # use boundaries from MarineRegions.com
    # union of world country boundaries and Exclusive Economic Zones (2014)
    # http://www.marineregions.org/downloads.php#unioneezcountry
    # Note: Kosovo is part of Serbia in EEZ file

    df = gpd.read_file(eez_file)
    df = df.rename(columns={'UNION': 'Country'})
    df['Country'] = df['Country'].replace({
        'Trinidad & Tobago': 'Trinidad and Tobago',
        'Bosnia & Herzegovina': 'Bosnia and Herzegovina',
        'Czech Republic': 'Czechia', # to match GFIT usage
        'Macedonia': 'North Macedonia', # to match GFIT usage
        'United States': 'USA',
        'East Timor': 'Timor-Leste',
        'Ivory Coast': "Cote d'Ivoire",
    })
    df = df.set_index('Country')

#     # create one blob for all world land and EEZ boundaries, using Shapely function cascaded_union 
#     # whatever is left out is, presumably, international waters
#     # this is used below to determine whether anything is in international waters
#     world_eez_and_land_boundaries = cascaded_union(df['geometry'])
    
    eez_and_land_boundaries = df
    return eez_and_land_boundaries #, world_eez_and_land_boundaries

In [356]:
eez_and_land_boundaries = read_eez_file(eez_file)

eez_4087 = eez_and_land_boundaries.to_crs('epsg:4087')

In [357]:
df = gas_pipes.copy()
df.replace('', float('nan'), inplace=True)
df = df[~df['Route'].isin(no_route_options)] # remove any routes that are capacity expansions only, etc.
df = df.dropna(subset=['Route']) # remove NaNs

df = df.reset_index(drop=True)

df['Route start'] = df['Route'].str.split(':').str[0]
df['Route_start_lat'] = df['Route start'].str.split(',').str[0].str.strip().astype(float)
df['Route_start_lon'] = df['Route start'].str.split(',').str[1].str.strip().astype(float)

df['Route end'] = df['Route'].str.split(':').str[-1]
df['Route_end_lat'] = df['Route end'].str.split(',').str[0].str.strip().astype(float)
df['Route_end_lon'] = df['Route end'].str.split(',').str[1].str.strip().astype(float)

gas_pipes_plus = df

route_starts = gpd.GeoDataFrame(
    gas_pipes_plus.set_index('ProjectID'), 
    geometry=gpd.points_from_xy(df.Route_start_lon, df.Route_start_lat)
).set_crs('epsg:4326')

route_starts_4087 = route_starts['geometry'].to_crs('epsg:4087')

route_ends = gpd.GeoDataFrame(
    gas_pipes_plus.set_index('ProjectID'), 
    geometry=gpd.points_from_xy(df.Route_end_lon, df.Route_end_lat)
).set_crs('epsg:4326')

route_ends_4087 = route_ends['geometry'].to_crs('epsg:4087')

In [358]:
# check the country polygons for validity
for country in eez_4087.index:
    eez = eez_4087.at[country, 'geometry']
    if eez.is_valid:
        pass
    else:
        print(f"For {country}, error: eez.is_valid = {eez.is_valid}")
        print(f"Excluded {country} from eez_4087")
        eez_4087 = eez_4087.drop(country)

#for i in eez_4087.index.values:
#    print(i)

In [359]:
def check_points_in_which_country(gas_pipes_plus, route_starts_4087, route_ends_4087, eez_4087):
    for row in gas_pipes_plus.index:
        project_id = gas_pipes_plus.at[row, 'ProjectID']
        pipeline_name = gas_pipes_plus.at[row, 'PipelineName']
        project_id = gas_pipes_plus.at[row, 'ProjectID']
        start_country = gas_pipes_plus.at[row, 'StartCountry']
        end_country = gas_pipes_plus.at[row, 'EndCountry']
        
        start_point_4087 = route_starts_4087.at[project_id]
        end_point_4087 = route_ends_4087.at[project_id]
        for point_type in ['start', 'end']:
            if point_type=='start':
                point_4087 = start_point_4087
            elif point_type=='end':
                point_4087 = end_point_4087
            # process start point first
            # comparing against both start country & end country
            for country in eez_4087.index:
                if country=='Kosovo':
                    # EEZ has it as part of Serbia
                    country_eez_4087 = eez_4087.at['Serbia', 'geometry']
                else:
                    country_eez_4087 = eez_4087.at[country, 'geometry']
                    
                within = False # initialize
                try:
                    within = point_4087.within(country_eez_4087)
                except:
                    pass
                if within==True:
                    if country not in [start_country, end_country]:
                        if country=='Alaska' and 'USA' in [start_country, end_country]:
                            pass
                        else:
                            dist_threshold = 5 # km
                            if start_country=='Kosovo':
                                start_country_eez = eez_4087.at['Serbia', 'geometry']
                            else:
                                start_country_eez = eez_4087.at[start_country, 'geometry']
                            start_dist_away_km = start_country_eez.distance(point_4087)/1000
                            if end_country=='Kosovo':
                                end_country_eez = eez_4087.at['Serbia', 'geometry']
                            else:
                                end_country_eez = eez_4087.at[end_country, 'geometry']
                            end_dist_away_km = end_country_eez.distance(point_4087)/1000
                            min_dist_away_km = min([start_dist_away_km, end_dist_away_km])
                            if min_dist_away_km < dist_threshold:
                                pass
                            else:
                                print(f"For {pipeline_name} ({project_id}):")
                                print(f"Start country (spreadsheet): {start_country}")
                                print(f"End country (spreadsheet): {end_country}")
                                if point_type=='start':
                                    coords = f"{gas_pipes_plus.at[row, 'Route_start_lat']}, {gas_pipes_plus.at[row, 'Route_start_lon']}"
                                elif point_type=='end':
                                    coords = f"{gas_pipes_plus.at[row, 'Route_end_lat']}, {gas_pipes_plus.at[row, 'Route_end_lon']}"
                                print(f"However, found the {point_type} point in EEZ of {country}, at: {coords}; minimum distance: {round(min_dist_away_km,1)}.")
                                route = gas_pipes_plus.at[row, 'Route']
                                if ';' in route:
                                    print("Warning, this pipeline route has separate segments; may be branching or not in order.")
                                print()

In [360]:
check_points_in_which_country(gas_pipes_plus, route_starts_4087, route_ends_4087, eez_4087)

For Nigeria-Morocco Gas Pipeline (P0463):
Start country (spreadsheet): Nigeria
End country (spreadsheet): Spain
However, found the end point in EEZ of Morocco, at: 35.7711602, -5.8579405; minimum distance: 20.0.
Warning, this pipeline route has separate segments; may be branching or not in order.

For Baltic Pipe Project (P0684):
Start country (spreadsheet): Denmark
End country (spreadsheet): Norway
However, found the start point in EEZ of Poland, at: 53.8, 15.2599; minimum distance: 86.2.
Warning, this pipeline route has separate segments; may be branching or not in order.

For Turkmenistan-Afghanistan-Pakistan-India Gas Pipeline (P0766):
Start country (spreadsheet): Turkmenistan
End country (spreadsheet): India
However, found the end point in EEZ of Afghanistan, at: 31.716949, 65.715115; minimum distance: 504.5.
Warning, this pipeline route has separate segments; may be branching or not in order.

For Hungary-Slovenia-Italy Interconnector Gas Pipeline (P1340):
Start country (spreadsh

In [361]:
def convert_gfit_to_linestring(coord_str, pipeline_name):
    '''
    Takes string from GFIT column of coordinates for a single pipeline,
    converts that string into Shapely LineString or MultiLinestring.
    '''

    if ':' in coord_str and ';' not in coord_str:
        # simple geometry; no branching
        # create nested list of lists, separating on colons        
        coord_list = coord_str.split(':')
        coord_list_tuples = []
        # non-branched pipeline (nested list with one level)
        # convert nested list of lists to list of tuples
        try:
            for element in coord_list:
                element_tuple = (float(element.split(',')[1]), 
                                 float(element.split(',')[0]))
                coord_list_tuples.append(element_tuple)
        except:
            print(f"Exception for {pipeline_name}; element: {element}") # for db
        route_conv = LineString(coord_list_tuples)

    elif ':' in coord_str and ';' in coord_str:
        # create a nested list of lists, separating on semicolons
        coord_list = coord_str.split(';')   
        # create a second level of nesting, separating on colons
        coord_list = [x.split(':') for x in coord_list]
        # branched pipeline (nested list with two levels)
        route_conv_ls_all = []
        
        for nested_list in coord_list:
            coord_list_tuples = []
            # process element
            try:
                for element in nested_list:
                    element_tuple = (float(element.split(',')[1]), 
                                     float(element.split(',')[0]))
                    coord_list_tuples.append(element_tuple)
            except:
                print(f"Exception for {pipeline_name}; element: {element}") # for db
            # process coord_list_tuples
            try:
                route_conv_ls = LineString(coord_list_tuples)
                route_conv_ls_all.append(route_conv_ls)
            except:
                print(f"Exception for {pipeline_name}; coord_list_tuples: {coord_list_tuples}") # for db
                pass
            
        route_conv = MultiLineString(route_conv_ls_all)
    
    #elif coord_str in no_route_options:      
    #    # create empty MultiLineString; no coordinates
    #    route_conv = MultiLineString([])
    #    
    #    print(f'No coordinates for {pipeline_name}: {coord_str}')
    #
    #else:
    #    # create empty MultiLineString; coordinates were missing or misformatted
    #    route_conv = MultiLineString([])
    #    
    #    print(f'Missing or misformatted coordinates for {pipeline_name}')
        
    return route_conv

In [362]:
def convert_all_pipelines(df):
    """
    Apply the conversion function to all pipelines in the dataframe.
    """
    # create geometry column with empty strings
    df['geometry'] = ''
    
    # filter to keep only pipelines with routes
    mask_route_1 = df['Route'].str.contains(',')
    mask_route_2 = df['Route'].str.contains(':')
    pipes_with_route = df.loc[(mask_route_1) & (mask_route_2)]
    
    for row in pipes_with_route.index:
        route_str = df.at[row, 'Route']
        pipeline_name = df.at[row, 'PipelineName']
        route_str_converted = convert_gfit_to_linestring(route_str, pipeline_name)
        df.at[row, 'geometry'] = route_str_converted
        
    return df

In [363]:
# code to create a dataframe with WKT formatted geometry

# (1) copy gas_pipelines, clean up
to_convert = gas_pipes.copy()
#to_convert = to_convert.dropna(subset=['Route'])
to_convert = to_convert[~to_convert['Route'].isin(no_route_options)]

# (2) convert all pipelines
gas_pipes_wkt = convert_all_pipelines(to_convert)
gas_pipes_wkt = gas_pipes_wkt.reset_index(drop=True)

# (3) store in a GeoDataFrame, attach a projection, transform to a different one
gas_pipes_wkt_gdf = gpd.GeoDataFrame(gas_pipes_wkt, geometry=gas_pipes_wkt['geometry'])
gas_pipes_wkt_gdf = gas_pipes_wkt_gdf.set_crs('epsg:4326')
gas_pipes_wkt_gdf_4087 = gas_pipes_wkt_gdf.to_crs('epsg:4087')

In [364]:
# calculate km of pipeline in each country in the EEZ database

all_countries = pd.DataFrame() # initialize

gdf_sel = gas_pipes_wkt_gdf_4087[['ProjectID', 'PipelineName', 'SegmentName', 'Wiki', 'geometry']]

for sel_country in eez_4087.index:
    one_country_overlay = overlay(
        gdf_sel, 
        eez_4087[eez_4087.index==sel_country][['geometry']], 
        how="intersection")

    one_country_overlay['length km in country'] = one_country_overlay.length / 1000
    one_country_overlay['Country'] = sel_country

    all_countries = all_countries.append(one_country_overlay, sort=False)

In [372]:
# print out mismatches between countries

df2 = gas_pipes_wkt.set_index('ProjectID')
df2 = df2.sort_values(by=['StartCountry', 'PipelineName', 'SegmentName'])

for project_id in df2.index:
    
    pipeline_name = df2.at[project_id, 'PipelineName']
    segment_name = df2.at[project_id, 'SegmentName']
    wiki_page = df2.at[project_id, 'Wiki']
    
    countries_list = df2.at[project_id, 'Countries'].split(',')
    countries_list = [x.strip() for x in countries_list]
    countries_list.sort()
    
    gis_sel = all_countries.copy()[all_countries['ProjectID']==project_id]
    gis_sel['length km in country'] = gis_sel['length km in country'].astype(int)
    
    # special handling for Alaska, Canary Islands, etc.
    # this tells our dataset to replace USA with Alaska, Kosovo with Serbia, etc. to match EEZ
    gis_sel['Country'] = gis_sel['Country'].replace({
        'Alaska': 'USA',
        'Canary Islands': 'Spain',
        #'Serbia': 'Kosovo' # till 2008 Serbia controlled Kosovo
        #'Kosovo': 'Serbia'
    })
    group_cols = ['ProjectID', 'PipelineName', 'SegmentName', 'Country']
    gis_sel = gis_sel.groupby(group_cols)[['length km in country']].sum().reset_index()
    
    # filter out small values
    km_threshold = 5
    gis_sel = gis_sel[gis_sel['length km in country']>km_threshold]
    
    gis_sel_countries = gis_sel['Country'].tolist()
    
    if set(countries_list) != set(gis_sel_countries):
        #print(f"{pipeline_name} {segment_name} ({project_id}) {wiki_page}")
        print(f"{pipeline_name} {segment_name} ({project_id}) - mismatch in countries")
        print(f"{wiki_page}")
        print(f"List in GFIT: {countries_list}")
        print(f"GIS analysis:")
        print(gis_sel[['Country', 'length km in country']])
        print()

Albania–Kosovo Gas Pipeline  (P3336) - mismatch in countries
https://www.gem.wiki/Albania%E2%80%93Kosovo_Gas_Pipeline
List in GFIT: ['Albania', 'Kosovo']
GIS analysis:
   Country  length km in country
0  Albania                   119
1   Serbia                   112

GALSI Pipeline  (P0697) - mismatch in countries
https://www.gem.wiki/GALSI_Pipeline
List in GFIT: ['Algeria', 'Italy']
GIS analysis:
   Country  length km in country
0  Algeria                   126
1   France                    64
2    Italy                   611
3  Tunisia                    35

Razlog - Bansko Gas Pipeline  (P2088) - mismatch in countries
https://www.gem.wiki/Razlog-Bansko_Gas_Pipeline
List in GFIT: ['Bulgaria']
GIS analysis:
Empty DataFrame
Columns: [Country, length km in country]
Index: []

Croatia-Slovenia Interconnector Gas Pipeline  (P1342) - mismatch in countries
https://www.gem.wiki/Croatia-Slovenia-Austria_Interconnector_Gas_Pipeline
List in GFIT: ['Croatia', 'Slovenia']
GIS analysis:
   Country

List in GFIT: ['Romania', 'Ukraine']
GIS analysis:
   Country  length km in country
1  Ukraine                    78

Nord Stream 2 Gas Pipeline  (P0752) - mismatch in countries
https://www.gem.wiki/Nord_Stream_2_Gas_Pipeline
List in GFIT: ['Germany', 'Russia']
GIS analysis:
   Country  length km in country
0  Denmark                   224
1  Finland                   701
2  Germany                   125
3   Russia                   207
4   Sweden                   647

Trans-Korea Gas Pipeline  (P1440) - mismatch in countries
https://www.gem.wiki/Trans-Korea_Gas_Pipeline
List in GFIT: ['Russia', 'South Korea']
GIS analysis:
       Country  length km in country
1  North Korea                   650
2       Russia                   162
3  South Korea                   346

Serbia-North Macedonia Interconnection Pipeline  (P2462) - mismatch in countries
https://www.gem.wiki/Serbia-North_Macedonia_Interconnector_Pipeline
List in GFIT: ['North Macedonia', 'Serbia']
GIS analysis:
  Country  

In [366]:
def print_missing_routes(df):
    """
    Print the pipelines with no route (empty in dataset)
    """
    print();print('===== MISSING ROUTES =====');print()
    count=0
    for index,row in df.iterrows():
        if row['Route']=='':
            print(row['PipelineName'], ' - ', row['Wiki'])
            count+=1
    if count==0:
        print('No routes are missing!')

In [367]:
print_missing_routes(gas_pipes)


===== MISSING ROUTES =====

PennEast Gas Pipeline  -  https://www.gem.wiki/PennEast_Gas_Pipeline
Port Arthur Gas Pipeline  -  https://www.gem.wiki/Port_Arthur_Gas_Pipeline
Yongan-Tongxiao Gas Pipeline 2  -  https://www.gem.wiki/Yongan-Tongxiao_Gas_Pipeline_2
Thi Vai Import Terminal Gas Pipeline  -  https://www.gem.wiki/Thi_Vai_Import_Terminal_Gas_Pipeline
South West LNG Terminal to Ca Mau Gas Pipeline  -  https://www.gem.wiki/South_West_LNG_Terminal_to_Ca_Mau_Gas_Pipeline
Son My LNG Terminal to Son My Electricity Center Gas Pipeline  -  https://www.gem.wiki/Son_My_LNG_Terminal_to_Son_My_Electricity_Center_Gas_Pipeline
Son My–Phu My Gas Pipeline  -  https://www.gem.wiki/Son_My%E2%80%93Phu_My_Gas_Pipeline
Son My–Su Tu Trang Gas Pipeline  -  https://www.gem.wiki/Son_My%E2%80%93Su_Tu_Trang_Gas_Pipeline
FSRU to Thai Binh Gas Pipeline  -  https://www.gem.wiki/FSRU_to_Thai_Binh_Gas_Pipeline
East to Southeast Gas Pipeline  -  https://www.gem.wiki/East_to_Southeast_Gas_Pipeline
Trans-ASEAN Gas

In [368]:
# TO DO: compare calculated length vs stated length. 
# Look into those with large discrepancies - over 50% difference

#BL:
#look at where calculated length 2x more than stated length - 90 total
#where route way too short too (half)
#some eia routes are very complex; need to bring in more complicated routes (e.g., in USA?)

In [369]:
# TO DO: Look for outlier values for length and diameter to try to catch errors.

In [370]:
# TO DO: Check capacity expansion projects: 
# Make sure all capacity expansion projects (which don’t involve laying more pipe) 
# have “capacity expansion only” in the column “Route”, which would make them not show up on the map. 
# Also make sure they don’t have a length listed.